In [ ]:
import pandas as pd
import lecilab_behavior_analysis.utils as utils
import lecilab_behavior_analysis.plots as plots
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import lecilab_behavior_analysis.df_transforms as dft
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import statsmodels.api as sm
%load_ext autoreload
%autoreload 2


In [ ]:
import warnings
warnings.filterwarnings('ignore')


single mouse

In [ ]:
# load data from cluster
tv_projects = utils.get_server_projects()
print(tv_projects)
# see the available animals
animals = utils.get_animals_in_project(tv_projects[1])
print(animals)
# download the data for a specific animal
mouse = "ACV002"
local_path = Path(utils.get_outpath()) / Path(tv_projects[1]) / Path("sessions") / Path(mouse)
# create the directory if it doesn't exist
local_path.mkdir(parents=True, exist_ok=True)
# download the session data
utils.rsync_session_data(
    project_name=tv_projects[1],
    animal=mouse,
    local_path=str(local_path),
    credentials=utils.get_idibaps_cluster_credentials(),
)
# load the data
df = pd.read_csv(local_path / Path(f'{mouse}.csv'), sep=";")

In [ ]:
# reduce the dataset to the psychometric version of the task
# Otherwise, we would include a lot of "easy" trials that would bias the fit
df_test = df[df["current_training_stage"] == "TwoAFC_visual_hard"]

psychometric curve 

In [ ]:
df_test = dft.get_performance_by_difficulty_ratio(df_test)
plots.psychometric_plot(df_test, x = 'visual_stimulus_ratio', y = 'first_choice_numeric')


In [ ]:
df_test = dft.get_performance_by_difficulty_diff(df_test)
plots.psychometric_plot(df_test, x = 'visual_stimulus_diff', y = 'first_choice_numeric', valueType = 'continue', )

GLM comparation

the following cell can be use to evaluate the model. It will be useful when comparing different models

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from scipy.optimize import minimize
import numpy as np

# Define the lapse logistic function with independent lapses for left and right
def lapse_logistic_independent(params, x, y):
    lapse_left, lapse_right, beta, x0 = params
    # Ensure lapse rates are within [0, 0.5]
    lapse_left = np.clip(lapse_left, 0, 0.5)
    lapse_right = np.clip(lapse_right, 0, 0.5)
    # Predicted probabilities
    p_left = lapse_left + (1 - lapse_left - lapse_right) / (1 + np.exp(-beta * (x - x0)))
    # Negative log-likelihood
    nll = -np.sum(y * np.log(p_left) + (1 - y) * np.log(1 - p_left))
    return nll

# Cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation
log_losses = []

# Perform cross-validation
for train_index, test_index in kf.split(df_test):
    # Split the data
    x_train, x_test = df_test['visual_stimulus_ratio'].values[train_index], df_test['visual_stimulus_ratio'].values[test_index]
    y_train, y_test = df_test['first_choice_numeric'].values[train_index], df_test['first_choice_numeric'].values[test_index]
    
    # Initial parameter guesses: [lapse_left, lapse_right, beta, x0]
    initial_params = [0.05, 0.05, 1, 0]
    
    # Fit the model on the training data
    result = minimize(
        lapse_logistic_independent,
        initial_params,
        args=(x_train, y_train),
        bounds=[(0, 0.5), (0, 0.5), (None, None), (None, None)]
    )
    
    # Extract fitted parameters
    lapse_left, lapse_right, beta, x0 = result.x
    
    # Generate predictions on the test data
    p_left_test = lapse_left + (1 - lapse_left - lapse_right) / (1 + np.exp(-beta * (x_test - x0)))
    
    # Calculate log loss for the test data
    loss = log_loss(y_test, p_left_test)
    log_losses.append(loss)

# Print cross-validation results
print(f"Cross-Validation Log Losses: {log_losses}")
print(f"Mean Log Loss: {np.mean(log_losses)}")
print(f"Standard ratio of Log Loss: {np.std(log_losses)}")

weight and stats for the different predictors:
- visual stimulus ratio (you call it deviation)
- visual stimulus diff. Nuo: change to "total intensity on left port"
- port where the animal is coming from
- interactions
- Nuo: add another regressor: the previous correct choice

We can play around with this things

In [ ]:
X = ['visual_stimulus_ratio',
    'previous_port_before_stimulus_numeric',
    'visual_ratio_diff_interact',
    'previous_left_choice_correct_numeric',
    'previous_right_choice_wrong_numeric',
    'previous_first_choice_numeric', 
    'visual_ratio_bright_interact', 
    'previous_last_choice_numeric'
        ]
df_new_for_fit = dft.parameters_for_fit(df_test)
results, model = utils.logi_model_fit(df_new_for_fit, X = X, y = 'first_choice_numeric')
print(results)

correct choice as output

In [ ]:
df_new_for_fit = dft.parameters_for_fit(df_test)
results = utils.logi_model_fit(df_new_for_fit, X = ['visual_stimulus_ratio',
                                                    'wrong_bright', 
                                                    # 'wrong_bright_zscore',
                                                    'previous_same_choice_correct_numeric', 
                                                    # 'previous_diff_choice_wrong_numeric', 
                                                    'previous_same_choice_numeric', 
                                                    'previous_correct_numeric'
                                                    ], y = 'correct_numeric')
print(results) 

In [ ]:
def previous_impact_on_time_kernel(series, max_lag=10, tau=5):
    kernel = np.exp(np.arange(1, max_lag+1) / tau)
    padded = np.concatenate([[0]*len(kernel), series])
    # time kernel convolve
    return np.array([
        np.dot(kernel, padded[i:i+len(kernel)])
        for i in range(len(series))
    ])

In [ ]:
"""The kernel stimates the weight subjects gives to each stimulus frame. It's usually computed via logistic regression
(https://en.wikipedia.org/wiki/Logistic_regression). We estimate the probability of a decision 'right' given some filters
(the betas or weights).
- p is the probability of choose right
- B0 isn't multiplied by any x and therefore is the bias. Normally is not included, but if the subject is biased, it's
best to do so. Bi are the weights of each frame, and there's one beta for each x
- x are the frames, there's one x for each B

In the wikipedia example plot, the x-axis would be the stimulus strength and the y-axis would be probability of
choose right. Then we fit the logistic regression curve. When we plot a kernel, what we're actually representing are
values of Bi. The values of beta can be computed in python with the 'logistic regression' from the 'sklearn' library
(https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).
- x is a matrix with my stimulus strengths (1 row per stimulus, one column for each frame, so 1*10)
- y is a vector with the subjects' choices
"""
endog = choices  # Your regressand (y)
exog = stimuli_frames (evidences)  # Your regressors (x)
model = sm.GLM(endog, exog, family=sm.families.Binomial(), missing='drop')  # GLM with Binomial family
results = model.fit()
params = results.params
beta_std_err = results.bse
p_values = results.pvalues
summary = results.summary()
print(summary)

plt.plot(params)  # The so-called kernel is just the plot of this weights (params)

auditory

In [ ]:
df_auditory = df[df['current_training_stage'] == 'TwoAFC_auditory_hard']

In [ ]:
X = np.array([get_timebin_evidence(eval(t)) for t in df_auditory['auditory_stimulus']])
df_aud_fit = dft.parameters_for_fit(df_auditory)
y = df_aud_fit['first_choice_numeric']
X_model = sm.add_constant(X) 
glm = sm.Logit(y, X_model).fit()
plt.plot(glm.params[1:])

In [ ]:
df_auditory = dft.get_performance_by_difficulty_ratio(df_auditory)
plots.psychometric_plot(df_auditory, x = 'total_evidence_strength', y = 'first_choice_numeric', valueType = 'continue')

In [ ]:
df_auditory_fit = dft.parameters_for_fit(df_auditory)

In [ ]:
results, _ = utils.logi_model_fit(df_auditory_fit, X = ['total_percentage_of_tones_left',
                      'number_of_tones_left',
                      'percentage_of_timebins_with_evidence_left', 
                      'total_evidence_strength', 
                      'amplitude_strength'
                          ], y = 'first_choice_numeric')
print(results)

Correct wrong psychometric curve

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
for i, linecolor in zip(df_new_for_fit[df_new_for_fit['previous_first_choice_numeric'] == 1].groupby('previous_correct_numeric'), ['red', 'green']):
    plots.psychometric_plot(df = i[1], 
                            x = 'visual_stimulus_ratio', 
                            y = 'first_choice_numeric', 
                            ax=ax[0],
                            point_kwargs={'marker': 'o', 'color': 'k', 'label': ''},
                            line_kwargs={'color': linecolor, 'label': 'previous ' + str(i[0])}
                                        )

for i, linecolor in zip(df_new_for_fit[df_new_for_fit['previous_first_choice_numeric'] == 0].groupby('previous_correct_numeric'), ['red', 'green']):
    plots.psychometric_plot(df = i[1], 
                            x = 'visual_stimulus_ratio', 
                            y = 'first_choice_numeric', 
                            ax=ax[1],
                            point_kwargs={'marker': 'o', 'color': 'k', 'label': ''},
                            line_kwargs={'color': linecolor, 'label': 'previous ' + str(i[0])}
                                        )
ax[0].legend()
ax[0].set_title("Left Choice Previous")
ax[1].legend()
ax[1].set_title("Right Choice Previous")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
for i, linecolor in zip(df_new_for_fit[df_new_for_fit['previous_correct_numeric'] == True].groupby('previous_first_choice_numeric'), ['gold', 'lightskyblue']):
    plots.psychometric_plot(df = i[1], 
                            x = 'visual_stimulus_ratio', 
                            y = 'first_choice_numeric', 
                            ax=ax[0],
                            point_kwargs={'marker': 'o', 'color': 'k', 'label': ''},
                            line_kwargs={'color': linecolor, 'label': 'previous ' + str(i[0])}
                        )

for i, linecolor in zip(df_new_for_fit[df_new_for_fit['previous_correct_numeric'] == False].groupby('previous_first_choice_numeric'), ['gold', 'lightskyblue']):
    plots.psychometric_plot(df = i[1], 
                            x = 'visual_stimulus_ratio', 
                            y = 'first_choice_numeric', 
                            ax=ax[1],
                            point_kwargs={'marker': 'o', 'color': 'k', 'label': ''},
                            line_kwargs={'color': linecolor, 'label': 'previous ' + str(i[0])}
                        )
ax[0].legend()
ax[0].set_title("Correct Choice Previous")
ax[1].legend()
ax[1].set_title("Incorrect Choice Previous")

Matrix format

In [ ]:
# let's use the absolute value of the lowest visual stimulus as a proxy for the brightness of the visual stimulus
df_test['visual_stimulus_lowest'] = df_test['visual_stimulus'].apply(lambda x: abs(eval(x)[0]) if eval(x)[0] < eval(x)[1] else abs(eval(x)[1]))
# create 10 bins for the absolute value of the lowest visual stimulus
min_value = df_test['visual_stimulus_lowest'].min()
max_value = df_test['visual_stimulus_lowest'].max()
bins = np.linspace(min_value, max_value, 11)
df_test['visual_stimulus_lowest_binned'] = pd.cut(df_test['visual_stimulus_lowest'], bins=bins, labels=[f"{b:.2f}" for b in bins[:-1]])
# create a pivot table with the visual stimulus ratio and absolute value of the lowest visual stimulus
pivot_table_abs = df_test.pivot_table(
    index='visual_stimulus_lowest_binned',
    columns='visual_stimulus_ratio',
    values='first_choice_numeric',
    aggfunc='mean',
    observed=True
)
# plot the heatmap
plt.figure(figsize=(5, 5))
sns.heatmap(pivot_table_abs, cmap='coolwarm', annot=True, fmt=".2f", cbar_kws={'label': 'Probability of Left Choice'})
plt.xlabel("Visual Stimulus ratio")
plt.ylabel("Absolute Value of Lowest Visual Stimulus")
plt.title("Heatmap of Probability of Left Choice")
# rotate the y-axis labels
plt.yticks(rotation=0)
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# transform visual_stimulus_lowest_binned to a numeric value for plotting
df_test['visual_stimulus_lowest_binned_num'] = pd.to_numeric(df_test['visual_stimulus_lowest_binned'], errors='coerce')

# make two plots, one for when the animals comes from the left and one for when it comes from the right
fig, axs = plt.subplots(1, 2, figsize=(12, 5), sharey=True)
# Plot for when the animal comes from the left
for ax, side in zip(axs.ravel(), ['left', 'right']):
    df_side = df_test[df_test['previous_port_before_stimulus'] == side]
    for i in df_side.groupby('visual_stimulus_ratio'):
        df_i = i[1].sort_values(by='visual_stimulus_lowest_binned_num')
        # drop nan
        df_i = df_i.dropna(subset=['visual_stimulus_lowest_binned_num'])
        X = df_i['visual_stimulus_lowest_binned_num'].values.reshape(-1, 1)
        y = df_i['first_choice_numeric'].values.astype(int)
        model = LogisticRegression()
        model.fit(X, y)
        y_pred = model.predict(X)
        y_prob = model.predict_proba(X)[:, 1]
        ax.plot(X, y_prob, label=f"Visual Stimulus ratio: {i[0]}")
    ax.set_xlabel("Absolute Value of Lowest Visual Stimulus")
    ax.set_ylabel("Probability of Left Choice")
    ax.legend()
    ax.set_title(f"Last Choice Before Stimulus: {side.capitalize()}")
plt.show()

Fit the lapse model independently considering previous choices

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

colors = ["blue", "orange"]

for color, side in zip(colors, ['left', 'right']):
    df_side = df_test[df_test['previous_port_before_stimulus'] == side]
    # Fit the model
    x = df_side['visual_stimulus_ratio'].values
    y = df_side['first_choice_numeric'].values
    result = minimize(
        lapse_logistic_independent,
        initial_params,
        args=(x, y),
        bounds=[(0, 0.5), (0, 0.5), (None, None), (None, None)]
    )

    # Extract fitted parameters
    lapse_left, lapse_right, beta, x0 = result.x
    print(f"Side: {side}, Lapse Left: {lapse_left}, Lapse Right: {lapse_right}, Slope (Beta): {beta}, PSE (x0): {x0}")

    # Generate predictions
    xs = np.linspace(df_side['visual_stimulus_ratio'].min(), df_side['visual_stimulus_ratio'].max(), 100)
    p_left = lapse_left + (1 - lapse_left - lapse_right) / (1 + np.exp(-beta * (xs - x0)))

    # Plot the fitted curve

    sns.pointplot(
        x='visual_stimulus_ratio',
        y='first_choice_numeric',
        data=df_side,
        estimator=lambda x: np.mean(x),
        color=color,
        markers='o',
        errorbar=("ci", 95),
        ax=ax,
        label=f'Choices when coming from {side}',
        native_scale=True,
        linestyles='',
    )
    ax.plot(xs, p_left, color=color, label='Lapse Logistic Fit')
    ax.set_xlabel("Visual Stimulus ratio")
    ax.set_ylabel("Probability of Left Choice")
    plt.title(f"Psychometric Curves")
    ax.legend()
plt.show()

I kept what you did for comparison here

In [ ]:
# It is interesting to compare the effects of the relative difference between the two visual stimuli,
# and the absolute difference between them.

# Maybe what we can do is to train another logistic regression model, adding as well the absolute difference
# between the two visual stimuli, and see how it affects the probability of a left choice.
# Do you know what I mean?

for i in df_test.groupby('visual_stimulus_ratio'):
    df_i = i[1].sort_values(by='visual_stimulus_diff')
    X = df_i['visual_stimulus_diff'].values.reshape(-1, 1)
    y = df_i['first_choice_numeric'].values.astype(int)
    model = LogisticRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    y_prob = model.predict_proba(X)[:, 1]
    plt.plot(X, y_prob, label=f"Visual Stimulus ratio: {i[0]}")
    plt.legend()
plt.xlabel("Visual Stimulus Difference")
plt.ylabel("Probability of Left Choice")
plt.show()

Multiple animals analysis


data extration

In [ ]:
df_dic = {}
for mouse in animals:
    local_path = Path(utils.get_outpath()) / Path(tv_projects[1]) / Path("sessions") / Path(mouse)
    # create the directory if it doesn't exist
    local_path.mkdir(parents=True, exist_ok=True)
    # download the session data
    utils.rsync_session_data(
        project_name=tv_projects[1],
        animal=mouse,
        local_path=str(local_path),
        credentials=utils.get_idibaps_cluster_credentials(),
    )
    # load the data
    df_dic[mouse] = pd.read_csv(local_path / Path(f'{mouse}.csv'), sep=";")

In [ ]:
df_dic_hard = {}
for df_name, df in zip(df_dic.keys(), df_dic.values()):
    if 'TwoAFC_visual_hard' in df["current_training_stage"].unique():
        df = df[df["current_training_stage"] == "TwoAFC_visual_hard"]
        df = dft.get_performance_by_difficulty_ratio(df)
        df = dft.get_performance_by_difficulty_diff(df)
        df_dic_hard[df_name] = df

In [ ]:
df_dic_hard_fit = {}
for df_name, df in zip(df_dic_hard.keys(), df_dic_hard.values()):
    df_hard_fit = df.copy(deep=True)
    df_hard_fit = dft.parameters_for_fit(df_hard_fit)
    df_dic_hard_fit[df_name] = df_hard_fit


In [ ]:
df_dic_hard_fit_firHalf = {}
for df_name, df in zip(df_dic_hard_fit.keys(), df_dic_hard_fit.values()):
    df_hard_fit_firHalf = df[:(len(df)//2)]
    df_dic_hard_fit_firHalf[df_name] = df_hard_fit_firHalf

In [ ]:
df_dic_hard_aud = {}
for df_name, df in zip(df_dic.keys(), df_dic.values()):
    if 'TwoAFC_auditory_hard' in df["current_training_stage"].unique():
        df = df[df["current_training_stage"] == "TwoAFC_auditory_hard"]
        df = dft.get_performance_by_difficulty_ratio(df)
        df_dic_hard_aud[df_name] = df

In [ ]:
df_dic_hard_aud_fit = {}
for df_name, df in zip(df_dic_hard_aud.keys(), df_dic_hard_aud.values()):
    df_hard_aud_fit = df.copy(deep=True)
    df_hard_aud_fit = dft.parameters_for_fit(df_hard_aud_fit)
    df_dic_hard_aud_fit[df_name] = df_hard_aud_fit

In [ ]:
plt.figure(figsize=(5, 5))
for df_name, df, color in zip(df_dic_hard.keys(), df_dic_hard.values(), sns.color_palette("colorblind", len(df_dic_hard))):
    plots.psychometric_plot(df, x='visual_stimulus_ratio', y='first_choice_numeric', point_kwargs={'color': color, 'label' : ''}, line_kwargs={'color': color, 'label': df_name})
plt.show()

In [ ]:
plt.figure(figsize=(5, 5))
for df_name, df, color in zip(df_dic_hard_aud.keys(), df_dic_hard_aud.values(), sns.color_palette("colorblind", len(df_dic_hard_aud))):
    plots.psychometric_plot(df, x='total_evidence_strength', y='first_choice_numeric', valueType = 'continue', point_kwargs={'color': color, 'label' : ''}, line_kwargs={'color': color, 'label': df_name})
plt.show()

In [ ]:
X = ['visual_stimulus_ratio',
    'visual_ratio_diff_interact',
    'visual_ratio_bright_interact', 
    'previous_left_choice_correct_numeric',
    'previous_right_choice_wrong_numeric',
    'previous_first_choice_numeric', 
    'previous_last_choice_numeric', 
    'previous_port_before_stimulus_numeric',
    # 'time_kernel_impact'
        ]
y = 'first_choice_numeric'
fig, ax = plt.subplots(2, 5, figsize=(20, 10))
for var, n in zip(X, range(len(X))):
    row = n // 5
    col = n % 5
    for df_name, df, color in zip(df_dic_hard_fit.keys(), df_dic_hard_fit.values(), sns.color_palette("colorblind", len(df_dic_hard_fit))):
        df_new = df.dropna(subset=[var, y]) 
        # Check if the variable in daraframe is discrete or continuous
        if df_new[var].nunique() < 10:
            # If discrete, plot as discrete
            plots.psychometric_plot(df = df_new,
                                x = var, 
                                y = y, 
                                ax = ax[row][col],
                                point_kwargs={'color': color, 'label': ''},
                                line_kwargs={'color': color, 'label': df_name}, 
                                valueType='discrete'
                                        )
        else:
            # If continuous, plot as continuous
            plots.psychometric_plot(df = df_new,
                                x = var, 
                                y = y, 
                                ax = ax[row][col],
                                point_kwargs={'color': color, 'label': ''},
                                line_kwargs={'color': color, 'label': df_name}, 
                                valueType='continue'
                                        )

find the optimal parameters of time kernel

In [ ]:
comb_dict = utils.verify_params_time_kernel(dic = df_dic_hard, y='first_choice_numeric')
sorted_items = sorted(comb_dict.items(), key=lambda item: abs(item[1]), reverse=True)
sorted_items[:5]

filter the correlated values

In [ ]:
X = ['visual_stimulus_ratio',
    'visual_ratio_diff_interact',
    'visual_ratio_bright_interact', 
    # 'previous_left_choice_correct_numeric',
    # 'previous_right_choice_wrong_numeric',
    # 'previous_first_choice_numeric', 
    # 'previous_last_choice_numeric', 
    # 'previous_port_before_stimulus_numeric',
    # 'time_kernel_impact'
        ]
corr_mat_list, norm_contribution_df = utils.filter_variables_for_model(dic_fit=df_dic_hard_fit_firHalf, X = X, y='first_choice_numeric', max_lag=19, tau=1)
plots.plot_filter_model_variables(corr_mat_list=corr_mat_list, norm_contribution_df=norm_contribution_df)

fit the model by filtered variables

In [ ]:

X = ['visual_stimulus_ratio',
    'previous_port_before_stimulus_numeric',
    'visual_ratio_diff_interact',
    'previous_left_choice_correct_numeric',
    # 'previous_right_choice_wrong_numeric',
    # 'previous_first_choice_numeric', 
    'visual_ratio_bright_interact', 
    # 'previous_last_choice_numeric', 
    'time_kernel_impact'
        ]
df_leftChoice_model_p = pd.DataFrame()
df_leftChoice_model_coef = pd.DataFrame()
df_leftChoice_model_z =  pd.DataFrame()
for df_name, df, color in zip(df_dic_hard_fit_firHalf.keys(), df_dic_hard_fit_firHalf.values(), sns.color_palette("colorblind", len(df_dic_hard_fit_firHalf))):
    df_for_fit = dft.parameters_for_fit(df)
    df_for_fit = dft.get_time_kernel_impact(df_for_fit, y='first_choice_numeric', max_lag=19, tau=1)
    _, model = utils.logi_model_fit(df_for_fit, X=X, y='first_choice_numeric')
    # plot the results
    df_leftChoice_model_p[df_name] = model.pvalues
    df_leftChoice_model_coef[df_name] = model.params
    df_leftChoice_model_z[df_name] = model.tvalues


compare the parameters of model by p, coef, z directly

In [ ]:
# plot the p-values coefficients and z
fig, ax = plt.subplots(3, 1, figsize=(15, 10))
df_leftChoice_model_p.index = ['intercept'] + X
df_leftChoice_model_coef.index = ['intercept'] + X
df_leftChoice_model_z.index = ['intercept'] + X
df_leftChoice_model_p_long = df_leftChoice_model_p.reset_index().melt(id_vars='index', var_name='Mouse', value_name='Value')
sns.boxplot(data=df_leftChoice_model_p_long, x='index', y='Value', ax=ax[0], color='lightgrey', boxprops=dict(alpha=0.3))
sns.scatterplot(data=df_leftChoice_model_p_long, ax=ax[0], x='index', y='Value', hue='Mouse', palette='colorblind')
ax[0].axhline(y=0.05, color='red', linestyle='--', label='Significance Threshold (0.05)')
df_leftChoice_model_coef_long = df_leftChoice_model_coef.reset_index().melt(id_vars='index', var_name='Mouse', value_name='Value')
sns.boxplot(data=df_leftChoice_model_coef_long, x='index', y='Value', ax=ax[1], color='lightgrey', boxprops=dict(alpha=0.3))
sns.scatterplot(data=df_leftChoice_model_coef_long, ax=ax[1], x='index', y='Value', hue='Mouse', palette='colorblind')
df_leftChoice_model_z_long = df_leftChoice_model_z.reset_index().melt(id_vars='index', var_name='Mouse', value_name='Value')
sns.boxplot(data=df_leftChoice_model_z_long, x='index', y='Value', ax=ax[2], color='lightgrey', boxprops=dict(alpha=0.3))
sns.scatterplot(data=df_leftChoice_model_z_long, ax=ax[2], x='index', y='Value', hue='Mouse', palette='colorblind')
# make x label not overlapping
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=8, ha='right')
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=8, ha='right')
ax[2].set_xticklabels(ax[2].get_xticklabels(), rotation=8, ha='right')
ax[0].set_xlabel("")
ax[1].set_xlabel("")
ax[2].set_xlabel("")
ax[0].set_ylabel("P-values")
ax[1].set_ylabel("Coefficients")
ax[2].set_ylabel("Z-scores")
plt.tight_layout()
plt.show()

correct choice model

In [ ]:
comb_dict = utils.verify_params_time_kernel(dic = df_dic_hard, y='correct_numeric')
sorted_items = sorted(comb_dict.items(), key=lambda item: abs(item[1]), reverse=True)
sorted_items[:5]

In [ ]:
X = ['abs_visual_stimulus_ratio',
    'wrong_bright', 
    'previous_same_choice_correct_numeric', 
    'previous_same_choice_numeric', 
    'previous_correct_numeric', 
    'previous_port_before_stimulus_numeric', 
    'previous_first_choice_numeric', 
    'previous_last_choice_numeric', 
    'time_kernel_impact'
        ]
corr_mat_list, norm_contribution_df = utils.filter_variables_for_model(dic_fit=df_dic_hard_fit_firHalf, X = X, y='correct_numeric', max_lag=19, tau=7)
plots.plot_filter_model_variables(corr_mat_list=corr_mat_list, norm_contribution_df=norm_contribution_df)

In [ ]:
import statsmodels.api as sm
from sklearn.metrics import r2_score

In [ ]:
def drop_one_var_contribution(df, x_cols, y_col, method='newton'):
    contributions = {var: [] for var in x_cols}
    X, y = utils.logi_model_fit_input(df, x_cols, y_col)
    model = sm.Logit(y, X).fit(method=method)
    r2 = r2_score(y, model.predict(X))
    for var in x_cols:
        X_reduced = X.drop(columns=[var])
        model_reduced = sm.Logit(y, X_reduced).fit(method=method)
        r2_reduced = r2_score(y, model_reduced.predict(X_reduced))
        delta = r2 - r2_reduced
        contributions[var].append(delta)
    # normalize the contributions
    avg_contrib = {var: np.mean(contrib) for var, contrib in contributions.items()}
    total = sum(avg_contrib.values())
    norm_contrib = {var: val / total for var, val in avg_contrib.items()}
    return pd.Series(norm_contrib)

In [ ]:
def filter_variables_for_model_2(dic_fit:dict, X:list, y:str, max_lag=None, tau=None):
    corr_mat_list = []
    norm_contribution_df = pd.DataFrame([])
    for df_name, df_for_fit in zip(dic_fit.keys(), dic_fit.values()):
        if (max_lag is not None) & (tau is not None):
            df_for_fit = dft.get_time_kernel_impact(df_for_fit, y=y, max_lag=max_lag, tau=tau)
        
        corr_fit_X_df = df_for_fit[X].corr()
        corr_mat_list.append(corr_fit_X_df)

        norm_contribution = drop_one_var_contribution(df_for_fit, x_cols = X, y_col = y, method='bfgs')
        norm_contribution_df[df_name] = norm_contribution

    return corr_mat_list, norm_contribution_df

In [ ]:
X = ['abs_visual_stimulus_ratio',
    'wrong_bright', 
    'previous_same_choice_correct_numeric', 
    'previous_same_choice_numeric', 
    'previous_correct_numeric', 
    'previous_port_before_stimulus_numeric', 
    'previous_first_choice_numeric', 
    'previous_last_choice_numeric', 
    'time_kernel_impact'
        ]
corr_mat_list, norm_contribution_df = filter_variables_for_model_2(dic_fit=df_dic_hard_fit_firHalf, X = X, y='correct_numeric', max_lag=19, tau=7)

plots.plot_filter_model_variables(corr_mat_list=corr_mat_list, norm_contribution_df=norm_contribution_df)

In [ ]:
X = ['abs_visual_stimulus_ratio',
    'wrong_bright', 
    'previous_same_choice_correct_numeric', 
    'previous_same_choice_numeric', 
    'previous_correct_numeric', 
    # 'previous_port_before_stimulus_numeric', 
    # 'previous_first_choice_numeric', 
    # 'previous_last_choice_numeric', 
    'time_kernel_impact'
        ]
corr_mat_list, norm_contribution_df = filter_variables_for_model_2(dic_fit=df_dic_hard_fit_firHalf, X = X, y='correct_numeric', max_lag=19, tau=7)

plots.plot_filter_model_variables(corr_mat_list=corr_mat_list, norm_contribution_df=norm_contribution_df)

In [ ]:
X = ['visual_stimulus_ratio',
    'wrong_bright', 
    'previous_same_choice_correct_numeric', 
    'previous_same_choice_numeric', 
    'previous_correct_numeric', 
    # 'previous_port_before_stimulus_numeric', 
    # 'previous_first_choice_numeric', 
    'previous_last_choice_numeric', 
    'time_kernel_impact'
    ]
df_correctChoice_model_p = pd.DataFrame()
df_correctChoice_model_coef = pd.DataFrame()
df_correctChoice_model_z =  pd.DataFrame()
for df_name, df, color in zip(df_dic_hard_fit_firHalf.keys(), df_dic_hard_fit_firHalf.values(), sns.color_palette("colorblind", len(df_dic_hard_fit_firHalf))):
    df_for_fit = dft.parameters_for_fit(df)
    df_for_fit = dft.get_time_kernel_impact(df_for_fit, y='correct_numeric', max_lag=19, tau=7)
    _, model = utils.logi_model_fit(df_for_fit, X=X, y='correct_numeric')
    # plot the results
    df_correctChoice_model_p[df_name] = model.pvalues
    df_correctChoice_model_coef[df_name] = model.params
    df_correctChoice_model_z[df_name] = model.tvalues

In [ ]:
# plot the p-values coefficients and z
fig, ax = plt.subplots(3, 1, figsize=(15, 10))
df_correctChoice_model_p.index = ['intercept'] + X
df_correctChoice_model_coef.index = ['intercept'] + X
df_correctChoice_model_z.index = ['intercept'] + X
df_correctChoice_model_p_long = df_correctChoice_model_p.reset_index().melt(id_vars='index', var_name='Mouse', value_name='Value')
sns.boxplot(data=df_correctChoice_model_p_long, x='index', y='Value', ax=ax[0], color='lightgrey', boxprops=dict(alpha=0.3))
sns.scatterplot(data=df_correctChoice_model_p_long, ax=ax[0], x='index', y='Value', hue='Mouse', palette='colorblind')
ax[0].axhline(y=0.05, color='red', linestyle='--', label='Significance Threshold (0.05)')
df_correctChoice_model_coef_long = df_correctChoice_model_coef.reset_index().melt(id_vars='index', var_name='Mouse', value_name='Value')
sns.boxplot(data=df_correctChoice_model_coef_long, x='index', y='Value', ax=ax[1], color='lightgrey', boxprops=dict(alpha=0.3))
sns.scatterplot(data=df_correctChoice_model_coef_long, ax=ax[1], x='index', y='Value', hue='Mouse', palette='colorblind')
df_correctChoice_model_z_long = df_correctChoice_model_z.reset_index().melt(id_vars='index', var_name='Mouse', value_name='Value')
sns.boxplot(data=df_correctChoice_model_z_long, x='index', y='Value', ax=ax[2], color='lightgrey', boxprops=dict(alpha=0.3))
sns.scatterplot(data=df_correctChoice_model_z_long, ax=ax[2], x='index', y='Value', hue='Mouse', palette='colorblind')
# make x label not overlapping
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=8, ha='right')
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=8, ha='right')
ax[2].set_xticklabels(ax[2].get_xticklabels(), rotation=8, ha='right')
ax[0].set_xlabel("")
ax[1].set_xlabel("")
ax[2].set_xlabel("")
ax[0].set_ylabel("P-values")
ax[1].set_ylabel("Coefficients")
ax[2].set_ylabel("Z-scores")
plt.tight_layout()
plt.show()

timebin evidence

In [ ]:
timebin_evidence_df = pd.DataFrame()
for df_name, df, color in zip(df_dic_hard_aud_fit.keys(), df_dic_hard_aud_fit.values(), sns.color_palette("colorblind", len(df_dic_hard_aud_fit))):
    X = np.array([utils.get_timebin_evidence(eval(t)) for t in df['auditory_stimulus']])
    y = df['first_choice_numeric']
    X_model = sm.add_constant(X) 
    glm = sm.Logit(y, X_model).fit()
    timebin_evidence_df[df_name] = glm.params[1:]

In [ ]:
fig = plt.figure(figsize=(12, 5))
timebin_evidence_df.index = range(len(timebin_evidence_df))
for df_name, col, color in zip(timebin_evidence_df.columns, timebin_evidence_df, sns.color_palette("colorblind", len(timebin_evidence_df))):
    plt.plot(timebin_evidence_df[col], color=color, label=df_name, linestyle='--', alpha=0.7)
plt.plot(timebin_evidence_df.mean(axis=1), color='black', label='Mean Coefficient', linewidth=2)
plt.xlabel("Time Bin")
plt.ylabel("Coefficient")
plt.title("Time Bin Evidence Coefficients for Left Choice")
plt.legend()

In [ ]:
timebin_evidence_df = pd.DataFrame()
for df_name, df, color in zip(df_dic_hard_aud_fit.keys(), df_dic_hard_aud_fit.values(), sns.color_palette("colorblind", len(df_dic_hard_aud_fit))):
    X = np.abs(np.array([utils.get_timebin_evidence(eval(t)) for t in df['auditory_stimulus']]))
    y = df['correct_numeric']
    X_model = sm.add_constant(X) 
    glm = sm.Logit(y, X_model).fit()
    timebin_evidence_df[df_name] = glm.params[1:]

In [ ]:
fig = plt.figure(figsize=(12, 5))
timebin_evidence_df.index = range(len(timebin_evidence_df))
for df_name, col, color in zip(timebin_evidence_df.columns, timebin_evidence_df, sns.color_palette("colorblind", len(timebin_evidence_df))):
    plt.plot(timebin_evidence_df[col], color=color, label=df_name, linestyle='--', alpha=0.7)
plt.plot(timebin_evidence_df.mean(axis=1), color='black', label='Mean Coefficient', linewidth=2)
plt.xlabel("Time Bin")
plt.ylabel("Coefficient")
plt.title("Time Bin Evidence Coefficients for Correct Choice")
plt.legend()

In [ ]:
comb_dict = utils.verify_params_time_kernel(dic = df_dic_hard_aud, y='first_choice_numeric')
sorted_items = sorted(comb_dict.items(), key=lambda item: abs(item[1]), reverse=True)
sorted_items[:5]

In [ ]:
X = ['total_percentage_of_tones_left',
    'number_of_tones_left',
    'percentage_of_timebins_with_evidence_left', 
    'total_evidence_strength', 
    'amplitude_strength',
    'previous_port_before_stimulus_numeric',
    'previous_left_choice_correct_numeric',
    'previous_right_choice_wrong_numeric',
    'previous_first_choice_numeric', 
    'previous_last_choice_numeric', 
    'time_kernel_impact'
        ]
corr_mat_list, norm_contribution_df = utils.filter_variables_for_model(dic=df_dic_hard_aud, X = X, y='first_choice_numeric', max_lag=2, tau=1)
plots.plot_filter_model_variables(corr_mat_list=corr_mat_list, norm_contribution_df=norm_contribution_df)

In [ ]:
X = [
    # 'total_percentage_of_tones_left',
    # 'number_of_tones_left',
    # 'percentage_of_timebins_with_evidence_left', 
    'total_evidence_strength', 
    'amplitude_strength',
    'previous_port_before_stimulus_numeric',
    # 'previous_left_choice_correct_numeric',
    'previous_right_choice_wrong_numeric',
    'previous_first_choice_numeric', 
    'previous_last_choice_numeric', 
    # 'time_kernel_impact'
    ]
df_leftChoice_model_aud_p = pd.DataFrame()
df_leftChoice_model_aud_coef = pd.DataFrame()
df_leftChoice_model_aud_z =  pd.DataFrame()
for df_name, df, color in zip(df_dic_hard_aud.keys(), df_dic_hard_aud.values(), sns.color_palette("colorblind", len(df_dic_hard_aud))):
    df_for_fit = dft.parameters_for_fit(df)
    df_for_fit = dft.get_time_kernel_impact(df_for_fit, y='first_choice_numeric', max_lag=2, tau=1)
    _, model = utils.logi_model_fit(df_for_fit, X=X, y='first_choice_numeric')
    # plot the results
    df_leftChoice_model_aud_p[df_name] = model.pvalues
    df_leftChoice_model_aud_coef[df_name] = model.params
    df_leftChoice_model_aud_z[df_name] = model.tvalues

In [ ]:
# plot the p-values coefficients and z
fig, ax = plt.subplots(3, 1, figsize=(15, 10))
df_leftChoice_model_aud_p.index = ['intercept'] + X
df_leftChoice_model_aud_coef.index = ['intercept'] + X
df_leftChoice_model_aud_z.index = ['intercept'] + X
df_leftChoice_model_aud_p_long = df_leftChoice_model_aud_p.reset_index().melt(id_vars='index', var_name='Mouse', value_name='Value')
sns.boxplot(data=df_leftChoice_model_aud_p_long, x='index', y='Value', ax=ax[0], color='lightgrey', boxprops=dict(alpha=0.3))
sns.scatterplot(data=df_leftChoice_model_aud_p_long, ax=ax[0], x='index', y='Value', hue='Mouse', palette='colorblind')
ax[0].axhline(y=0.05, color='red', linestyle='--', label='Significance Threshold (0.05)')
df_leftChoice_model_aud_coef_long = df_leftChoice_model_aud_coef.reset_index().melt(id_vars='index', var_name='Mouse', value_name='Value')
sns.boxplot(data=df_leftChoice_model_aud_coef_long, x='index', y='Value', ax=ax[1], color='lightgrey', boxprops=dict(alpha=0.3))
sns.scatterplot(data=df_leftChoice_model_aud_coef_long, ax=ax[1], x='index', y='Value', hue='Mouse', palette='colorblind')
df_leftChoice_model_aud_z_long = df_leftChoice_model_aud_z.reset_index().melt(id_vars='index', var_name='Mouse', value_name='Value')
sns.boxplot(data=df_leftChoice_model_aud_z_long, x='index', y='Value', ax=ax[2], color='lightgrey', boxprops=dict(alpha=0.3))
sns.scatterplot(data=df_leftChoice_model_aud_z_long, ax=ax[2], x='index', y='Value', hue='Mouse', palette='colorblind')
# make x label not overlapping
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=8, ha='right')
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=8, ha='right')
ax[2].set_xticklabels(ax[2].get_xticklabels(), rotation=8, ha='right')
ax[0].set_xlabel("")
ax[1].set_xlabel("")
ax[2].set_xlabel("")
ax[0].set_ylabel("P-values")
ax[1].set_ylabel("Coefficients")
ax[2].set_ylabel("Z-scores")
plt.tight_layout()
plt.show()

model for auditory stimulus

In [ ]:
fig, ax = plt.subplots(2, 5, figsize=(20, 10))
for df_name, df, n in zip(df_dic_hard.keys(), df_dic_hard.values(), range(len(df_dic_hard))):
    row = n // 5
    col = n % 5
    for session, color in zip(df['session'].unique(), sns.color_palette("crest", len(df['session'].unique()))):
        df_session = df[df['session'] == session]
        plots.psychometric_plot(df_session, x='visual_stimulus_ratio', y='left_choice',ax=ax[row, col], point_kwargs={'color': color, 'label' : ''}, line_kwargs={'color': color, 'label': ''})
    ax[row, col].set_title(f"Psychometric Curve for {df_name}")
plt.tight_layout()
# Add a colorbar to indicate the session
cbar = plt.colorbar(plt.cm.ScalarMappable(cmap=sns.color_palette("crest", as_cmap=True)), orientation='horizontal', ax=ax, shrink=0.3)
cbar.set_ticks([])
cbar.set_label('before → after')
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 10))
for df_name, df, color in zip(df_dic_hard.keys(), df_dic_hard.values(), sns.color_palette("colorblind", len(df_dic_hard))):
    # divide into different groups with 1000 trials
    df['trial_group'] = np.arange(len(df)) // 1000
    lapse_left = []
    lapse_right = []
    slope = []
    bias = []
    for group in df['trial_group'].unique():
        df_group = df[df['trial_group'] == group]
        pleft, params = utils.fit_lapse_logistic_independent(df_group['visual_stimulus_diff'], df_group['left_choice'])
        # params in fit_lapse_logistic_independent = (lapse_left, lapse_right, beta, x0)
        lapse_left.append(params[0])
        lapse_right.append(params[1])
        slope.append(params[2])
        bias.append(params[3])
    ax[0, 0].plot(lapse_left, c=color, label=df_name)
    ax[0, 1].plot(lapse_right, c=color, label=df_name)
    ax[1, 0].plot(slope, c=color, label=df_name)
    ax[1, 1].plot(bias, c=color, label=df_name)
ax[0, 0].set_ylabel("Lapse Left")
ax[0, 1].set_ylabel("Lapse Right")
ax[1, 0].set_ylabel("slope")
ax[1, 1].set_ylabel("bias")
for ax1 in ax.flat:
    ax1.set_xlabel('1000_trials')
    ax1.legend()
plt.suptitle("Model for left choice on visual stimulus ratio")
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 10))
for df_name, df, color in zip(df_dic_hard_aud.keys(), df_dic_hard_aud.values(), sns.color_palette("colorblind", len(df_dic_hard_aud))):
    # divide into different groups with 1000 trials
    df['trial_group'] = np.arange(len(df)) // 1000
    lapse_left = []
    lapse_right = []
    slope = []
    bias = []
    for group in df['trial_group'].unique():
        df_group = df[df['trial_group'] == group]
        pleft, params = utils.fit_lapse_logistic_independent(df_group['total_evidence_strength'], df_group['first_choice_numeric'])
        # params in fit_lapse_logistic_independent = (lapse_left, lapse_right, beta, x0)
        lapse_left.append(params[0])
        lapse_right.append(params[1])
        slope.append(params[2])
        bias.append(params[3])
    ax[0, 0].plot(lapse_left, c=color, label=df_name)
    ax[0, 1].plot(lapse_right, c=color, label=df_name)
    ax[1, 0].plot(slope, c=color, label=df_name)
    ax[1, 1].plot(bias, c=color, label=df_name)
ax[0, 0].set_ylabel("Lapse Left")
ax[0, 1].set_ylabel("Lapse Right")
ax[1, 0].set_ylabel("slope")
ax[1, 1].set_ylabel("bias")
for ax1 in ax.flat:
    ax1.set_xlabel('1000_trials')
    ax1.legend()
plt.suptitle("Model for left choice on auditory evidence strenth")
plt.tight_layout()
plt.show()


In [ ]:
X = ['visual_stimulus_ratio',
    'previous_port_before_stimulus_numeric',
    'visual_ratio_diff_interact',
    'previous_left_choice_correct_numeric',
    'previous_right_choice_wrong_numeric',
    'previous_first_choice_numeric', 
    'visual_ratio_bright_interact', 
    'previous_last_choice_numeric'
        ]
dic_leftChoice_model_p = {}
dic_leftChoice_model_coef = {}
dic_leftChoice_model_z = {}
for df_name, df, n in zip(df_dic_hard.keys(), df_dic_hard.values(), range(len(df_dic_hard))):
    df_leftChoice_model_p = pd.DataFrame()
    df_leftChoice_model_coef = pd.DataFrame()
    df_leftChoice_model_z =  pd.DataFrame()
    # create a session group column
    unique_sessions = df['session'].unique()
    session_to_group = {session: i // 10 for i, session in enumerate(unique_sessions)} # this groups sessions in groups of 10
    df['session_group'] = df['session'].map(session_to_group)
    for session, color in zip(df['session_group'].unique(), sns.color_palette("crest", len(df['session'].unique()))):
        df_session = df[df['session_group'] == session]
        df_session_for_fit = dft.parameters_for_fit(df_session)
        # # normalize the X columns in the df_session_for_fit by zscore
        # df_session_for_fit[X] = df_session_for_fit[X].apply(lambda x: (x - x.mean()) / x.std())
        _, model = utils.logi_model_fit(df_session_for_fit, X=X, y='first_choice_numeric', method='powell')
        df_leftChoice_model_p[session] = model.pvalues
        df_leftChoice_model_coef[session] = model.params
        df_leftChoice_model_z[session] = model.tvalues
    df_leftChoice_model_p.index = ['intercept'] + X
    df_leftChoice_model_coef.index = ['intercept'] + X
    df_leftChoice_model_z.index = ['intercept'] + X
    dic_leftChoice_model_p[df_name] = df_leftChoice_model_p.T
    dic_leftChoice_model_coef[df_name] = df_leftChoice_model_coef.T
    dic_leftChoice_model_z[df_name] = df_leftChoice_model_z.T

In [ ]:
fig, ax = plt.subplots(3, len(X)+1, figsize=(30, 15))
for n, param in zip(range(len(X)+1), ['intercept'] + X):
    for mouse, color in zip(dic_leftChoice_model_p, sns.color_palette("colorblind", len(dic_leftChoice_model_p))):
        ax[0, n].plot(dic_leftChoice_model_p[mouse].index, dic_leftChoice_model_p[mouse].iloc[:, n], label=mouse, color=color)
    ax[0, n].set_title(param)
    ax[0, n].set_xlabel("10_Session")
    ax[0, n].set_ylabel("p-value")
    ax[0, n].axhline(y=0.05, color='red', linestyle='--', label='Significance 0.05')
    ax[0, n].legend()
for n, param in zip(range(len(X)+1), ['intercept'] + X):
    for mouse, color in zip(dic_leftChoice_model_coef, sns.color_palette("colorblind", len(dic_leftChoice_model_coef))):
        ax[1, n].plot(dic_leftChoice_model_coef[mouse].index, dic_leftChoice_model_coef[mouse].iloc[:, n], label=mouse, color=color)
    ax[1, n].set_title(param)
    ax[1, n].set_xlabel("10_Session")
    ax[1, n].set_ylabel("Coefficient")
    ax[1, n].legend()
for n, param in zip(range(len(X)+1), ['intercept'] + X):
    for mouse, color in zip(dic_leftChoice_model_z, sns.color_palette("colorblind", len(dic_leftChoice_model_z))):
        ax[2, n].plot(dic_leftChoice_model_z[mouse].index, dic_leftChoice_model_z[mouse].iloc[:, n], label=mouse, color=color)
    ax[2, n].set_title(param)
    ax[2, n].set_xlabel("10_Session")
    ax[2, n].set_ylabel("Z-score")
    ax[2, n].legend()

In [ ]:
X = ['visual_stimulus_ratio',
    'wrong_bright', 
    # 'wrong_bright_zscore',
    'previous_same_choice_correct_numeric', 
    # 'previous_diff_choice_wrong_numeric', 
    'previous_same_choice_numeric', 
    'previous_correct_numeric'
    'roa_choice_numeric'
    ]
dic_correctChoice_model_p = {}
dic_correctChoice_model_coef = {}
dic_correctChoice_model_z = {}
for df_name, df, n in zip(df_dic_hard.keys(), df_dic_hard.values(), range(len(df_dic_hard))):
    row = n // 5
    col = n % 5
    df_correctChoice_model_p = pd.DataFrame()
    df_correctChoice_model_coef = pd.DataFrame()
    df_correctChoice_model_z =  pd.DataFrame()
    # create a session group column
    unique_sessions = df['session'].unique()
    session_to_group = {session: i // 10 for i, session in enumerate(unique_sessions)} # this groups sessions in groups of 10
    df['session_group'] = df['session'].map(session_to_group)
    for session, color in zip(df['session_group'].unique(), sns.color_palette("crest", len(df['session'].unique()))):
        df_session = df[df['session_group'] == session]
        df_session_for_fit = dft.parameters_for_fit(df_session)
        # # normalize the X columns in the df_session_for_fit by zscore
        # df_session_for_fit[X] = df_session_for_fit[X].apply(lambda x: (x - x.mean()) / x.std())
        _, model = utils.logi_model_fit(df_session_for_fit, X=X, y='correct_numeric', method='powell')
        df_correctChoice_model_p[session] = model.pvalues
        df_correctChoice_model_coef[session] = model.params
        df_correctChoice_model_z[session] = model.tvalues
    df_correctChoice_model_p.index = ['intercept'] + X
    df_correctChoice_model_coef.index = ['intercept'] + X
    df_correctChoice_model_z.index = ['intercept'] + X
    dic_correctChoice_model_p[df_name] = df_correctChoice_model_p.T
    dic_correctChoice_model_coef[df_name] = df_correctChoice_model_coef.T
    dic_correctChoice_model_z[df_name] = df_correctChoice_model_z.T

In [ ]:
fig, ax = plt.subplots(3, len(X)+1, figsize=(30, 15))
for n, param in zip(range(len(X)+1), ['intercept'] + X):
    for mouse, color in zip(dic_correctChoice_model_p, sns.color_palette("colorblind", len(dic_correctChoice_model_p))):
        ax[0, n].plot(dic_correctChoice_model_p[mouse].index, dic_correctChoice_model_p[mouse].iloc[:, n], label=mouse, color=color)
    ax[0, n].set_title(param)
    ax[0, n].set_xlabel("10_Session")
    ax[0, n].set_ylabel("p-value")
    ax[0, n].axhline(y=0.05, color='red', linestyle='--', label='Significance 0.05')
    ax[0, n].legend()
for n, param in zip(range(len(X)+1), ['intercept'] + X):
    for mouse, color in zip(dic_correctChoice_model_coef, sns.color_palette("colorblind", len(dic_correctChoice_model_coef))):
        ax[1, n].plot(dic_correctChoice_model_coef[mouse].index, dic_correctChoice_model_coef[mouse].iloc[:, n], label=mouse, color=color)
    ax[1, n].set_title(param)
    ax[1, n].set_xlabel("10_Session")
    ax[1, n].set_ylabel("Coefficient")
    ax[1, n].legend()
for n, param in zip(range(len(X)+1), ['intercept'] + X):
    for mouse, color in zip(dic_correctChoice_model_z, sns.color_palette("colorblind", len(dic_correctChoice_model_z))):
        ax[2, n].plot(dic_correctChoice_model_z[mouse].index, dic_correctChoice_model_z[mouse].iloc[:, n], label=mouse, color=color)
    ax[2, n].set_title(param)
    ax[2, n].set_xlabel("10_Session")
    ax[2, n].set_ylabel("Z-score")
    ax[2, n].legend()